In [ ]:
import librosa
from librosa import display
import matplotlib.pyplot as plt
import scipy
import os
import tensorflow as tf
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Dense
from keras.losses import binary_crossentropy
from keras.activations import sigmoid, relu
from keras.models import Sequential
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from mido import MidiFile
import pandas as pd
import numpy as np
import csv
import multiprocessing as mp
import shutil


In [ ]:
sourcePath = "C:\\Users\\Gebruiker\\Desktop\\Database Tabs Wav\\train\\wav\\"
targetPath = "C:\\Users\\Gebruiker\\Desktop\\testfolder\\"


# Data preprocessing IMAGES

In [ ]:
def generateSpectogram(sourcePath,targetPath):     #generate Mel-spectogram from wav 
    
    plt.box(0)
    base, ext= os.path.splitext(file) #saving generated spectogram in target directory
    samples, sampling_rate = librosa.load(sourcePath)
    spectogram = librosa.stft(samples)
    mel = librosa.feature.melspectrogram(y=samples,sr=sampling_rate,S=spectogram,n_fft=2048,hop_length=512,n_mels=128)
    mel_dB = librosa.amplitude_to_db(np.abs(mel)) 
    librosa.display.specshow(mel_dB,cmap='gray')
    plt.savefig(targetPath  + f"\\{base}" + ".png")
    plt.close()

# Transforming wav files to spectograms

In [ ]:
for file in os.listdir(sourcePath):
    generateSpectogram(sourcePath+ f"\\{file}",targetPath)  #generating spectogram
       

# Data preprocessing LABELS

# method 1 - flow from Dataframe


In [ ]:
file_path = r"C:\Users\Gebruiker\Desktop\nsynth-train\examples.json"                       #transposing(switching) rows-columns
json_file = pd.read_json(file_path)
json_file_transposed = json_file.transpose()
json_file_transposed.head()
json_file_transposed.to_csv(r"C:\Users\Gebruiker\Desktop\transposed_jsonfile.csv")           

with open(r"C:\Users\Gebruiker\Desktop\Pitch-labels_GUITARS.csv",'w',newline='') as labels_file_guitar: #trimming down data
    writeData = csv.writer(labels_file_guitar,delimiter=',')
    writeData.writerow(['',"pitch"])
    
    with open(r"C:\Users\Gebruiker\Desktop\Pitch-labels_ALL.csv",'r') as labels_file:
        readData =  csv.reader(labels_file,delimiter=',')

        for row in readData:
            if(("guitar" in row[0]) == 1):
                writeData.writerow(row)
   
            
            
with open(path_directory + '\labels.csv','r') as csv_file:                #appending .png to all filenames
    row_reader = csv.reader(csv_file,delimiter = ',')
    with open(r'C:\Users\Gebruiker\Desktop\edited_labels.csv','w',newline='') as csv_edited:
        row_writer = csv.writer(csv_edited,delimiter = ',')
        for row in row_reader:
      
            row_writer.writerow([row[0] + '.png'] )
            print(row[0] + '.png')  





ONE HOT ENCODING pitch labels

In [ ]:
headerRow2 = ['filename','A0','A#0','B0','C1','C#1','D1','D#1','E1','F1',
         'F#1','G1','G#1','A1','A#1','B1','C2','C#2','D2','D#2','E2','F2',
         'F#2','G2','G#2','A2','A#2','B2','C3','C#3','D3','D#3','E3','F3',
         'F#3','G3','G#3','A3','A#3','B3','C4','C#4','D4','D#4','E4','F4',
         'F#4','G4','G#4','A4','A#4','B4','C5','C#5','D5','D#5','E5','F5',
         'F#5','G5','G#5','A5','A#5','B5','C6','C#6','D6','D#6','E6','F6',
         'F#6','G6','G#6','A6','A#6','B6','C7','C#7','D7','D#7','E7','F7',
         'F#7','G7','G#7','A7','A#7','B7','C8']
print(headerRow2)

In [ ]:
with open(r"C:\Users\Gebruiker\Desktop\labels_onehotencoded1.csv",'r') as labels_onehotencoded1:
    readData = csv.reader(labels_onehotencoded1, delimiter = ',')
    headerRow1 = next(readData)
    
    with open(r"C:\Users\Gebruiker\Desktop\labels_onehotencoded2.csv",'w',newline='') as labels_onehotencoded2:
        writeData = csv.writer(labels_onehotencoded2)
        writeData.writerow(headerRow2)
        for row in readData:
            placeholderRow = []
           
            for index, value in enumerate(headerRow1[1: len(headerRow1)],start = 1):
                
                if(row[1] == headerRow1[index]):
                    placeholderRow.append(1)
                else:
                    placeholderRow.append(0)
                    
            writeData.writerow(row[0:1] + placeholderRow)

                 

 TASK trim the labels even further down to contain only the 19996 labels for spectogram


In [76]:
listSpectogram = []                      #see if label names exist in spectogram - if yes - keep

for file in os.listdir(r'C:\Users\Gebruiker\Desktop\training\images'):  
    listSpectogram.append(file)
    
with open(r"C:\Users\Gebruiker\Desktop\labels_onehotencoded2.csv",'r') as file1:
    readData = csv.reader(file1,delimiter =",")
    headerRow = next(readData)
        
    with open(r"C:\Users\Gebruiker\Desktop\labels_onehotencodedFINAL.csv",'w',newline='') as file2:
        writeData = csv.writer(file2,delimiter = ',')
        writeData.writerow(headerRow)
        headerRow1 = next(readData)
        for row in readData:
            if(row[0] in listSpectogram):
                writeData.writerow(row)

